Вспомним решаемую проблему:
$$
\boldsymbol{x}(t) = \exp\left\{ \hat{A}t + \sum\limits_i \hat{B}_i W_i \right\}\boldsymbol{x}_0
$$
На основе формулы Ито конструируем стохастическое дифференциальное уравнение с таким решением:
$$
d\boldsymbol{x} = \left(\hat{A} + \dfrac{1}{2} \sum\limits_{i}\hat{B}_i^2 \right)\boldsymbol{x} dt + \sum\limits_i \hat{B}_i\boldsymbol{x}dW_i  
$$
Положим здесь:
$$
\hat{A} = \begin{pmatrix}
-\frac{1}{10} & -2\\
2 & -\frac{1}{10}
\end{pmatrix},  \qquad \hat{B}_1 = \begin{pmatrix}
0 & -0.1\\
0.1 & -0.1
\end{pmatrix},  \qquad \hat{B}_2 = \begin{pmatrix}
0 & -0.2\\
-0.2 & 0.3
\end{pmatrix}
$$

In [1]:
from sympy import *

In [84]:
g, w = symbols('gamma, omega', real=True)

A = Matrix([[g, -w],[w, g]])
X = MatrixSymbol('X', 2, 1)
t, dW1, dW2, dt = symbols("t, dW1, dW2, dt", real=True)
B1 = MatrixSymbol("B[1]", 2, 2)
B2 = MatrixSymbol("B[2]", 2, 2)
r = symbols('r', positive=True)
phi = symbols("phi")
RF = Matrix([r, phi])

In [87]:
dX = (A + Rational(1,2)*(B1**2))*X*dt + B1*X*dW1

In [88]:
dX.as_explicit()

Matrix([
[dW1*(B[1][0, 0]*X[0, 0] + B[1][0, 1]*X[1, 0]) + dt*((gamma + B[1][0, 0]**2/2 + B[1][0, 1]*B[1][1, 0]/2)*X[0, 0] + (-omega + B[1][0, 0]*B[1][0, 1]/2 + B[1][0, 1]*B[1][1, 1]/2)*X[1, 0])],
[ dW1*(B[1][1, 0]*X[0, 0] + B[1][1, 1]*X[1, 0]) + dt*((gamma + B[1][0, 1]*B[1][1, 0]/2 + B[1][1, 1]**2/2)*X[1, 0] + (omega + B[1][0, 0]*B[1][1, 0]/2 + B[1][1, 0]*B[1][1, 1]/2)*X[0, 0])]])

In [89]:
def ItoDiff(f, t, X, dt, dX):
    result = f.diff(t)*dt
    N = shape(X)[0]
    for i in range(N):
        result = result + f.diff(X[i])*dX[i]
        for j in range(N):
            result = result + Rational(1,2)*f.diff(X[i], X[j])*dX[i]*dX[j]
    return result

Some tests:

In [90]:
Y = MatrixSymbol("Y", 2, 1)
dY = Matrix([dW1 - 2*dW2, -dW1-3*dW2])
f = Y[0]**2 + Y[1]**2

In [91]:
ItoDiff(f, t, Y, dt, dY)

(-dW1 - 3*dW2)**2 + 2*(-dW1 - 3*dW2)*Y[1, 0] + (dW1 - 2*dW2)**2 + 2*(dW1 - 2*dW2)*Y[0, 0]

In [109]:
rfunc = sqrt(X[0]**2 + X[1]**2)
phifunc = atan2(X[1], X[0])
dWsubs = {dW1**2:dt, dW2**2:dt, dW1*dW2:0, dt**2:0, dW1*dt:0, dW2*dt:0}
Xsubs = {X[0]:RF[0]*cos(RF[1]), X[1]:RF[0]*sin(RF[1])}

In [126]:
dr = expand(ItoDiff(rfunc, t, X, dt, dX)).subs(dWsubs).subs(Xsubs)

In [127]:
dr = trigsimp(dr)

In [128]:
dr = powsimp(dr)

In [129]:
dr

dW1*r*(1 - cos(2*phi))*B[1][1, 1]/2 + dW1*r*(cos(2*phi) + 1)*B[1][0, 0]/2 + dW1*r*sin(2*phi)*B[1][0, 1]/2 + dW1*r*sin(2*phi)*B[1][1, 0]/2 + dt*gamma*r - dt*r*(1 - cos(2*phi))**2*B[1][1, 1]**2/8 + dt*r*(1 - cos(2*phi))*B[1][0, 1]**2/4 + dt*r*(1 - cos(2*phi))*B[1][0, 1]*B[1][1, 0]/4 + dt*r*(1 - cos(2*phi))*B[1][1, 1]**2/2 + dt*r*(-2*sin(2*phi) + sin(4*phi))*B[1][0, 1]*B[1][1, 1]/8 + dt*r*(-2*sin(2*phi) + sin(4*phi))*B[1][1, 0]*B[1][1, 1]/8 - dt*r*(2*sin(2*phi) + sin(4*phi))*B[1][0, 0]*B[1][0, 1]/8 - dt*r*(2*sin(2*phi) + sin(4*phi))*B[1][0, 0]*B[1][1, 0]/8 - dt*r*(cos(2*phi) + 1)**2*B[1][0, 0]**2/8 + dt*r*(cos(2*phi) + 1)*B[1][0, 0]**2/2 + dt*r*(cos(2*phi) + 1)*B[1][0, 1]*B[1][1, 0]/4 + dt*r*(cos(2*phi) + 1)*B[1][1, 0]**2/4 + dt*r*(cos(4*phi) - 1)*B[1][0, 0]*B[1][1, 1]/8 + dt*r*(cos(4*phi) - 1)*B[1][0, 1]**2/16 + dt*r*(cos(4*phi) - 1)*B[1][0, 1]*B[1][1, 0]/8 + dt*r*(cos(4*phi) - 1)*B[1][1, 0]**2/16 + 3*dt*r*sin(2*phi)*B[1][0, 0]*B[1][0, 1]/4 + dt*r*sin(2*phi)*B[1][0, 0]*B[1][1, 0]/4 + dt*

In [160]:
dphi = expand(ItoDiff(phifunc, t, X, dt, dX)).subs(dWsubs).subs(Xsubs)

In [162]:
dphi = trigsimp(dphi)

In [163]:
dphi = powsimp(dphi)

In [164]:
dphi = expand(dphi)

In [175]:
collect(dphi.coeff(dt), [cos(2*phi), sin(2*phi), cos(4*phi), sin(4*phi)])

omega + (-B[1][0, 0]*B[1][1, 0]/4 + B[1][0, 1]*B[1][1, 1]/4)*cos(2*phi)**2 + (B[1][0, 1]**2/4 - B[1][1, 0]**2/4)*sin(2*phi) + (-B[1][0, 0]*B[1][0, 1]/4 - B[1][0, 0]*B[1][1, 0]/8 + B[1][0, 1]*B[1][1, 1]/8 + B[1][1, 0]*B[1][1, 1]/4)*cos(4*phi) + (B[1][0, 0]*B[1][0, 1]/4 - B[1][0, 0]*B[1][1, 0]/4 - B[1][0, 1]*B[1][1, 1]/4 + B[1][1, 0]*B[1][1, 1]/4)*cos(2*phi) + (B[1][0, 0]**2/8 - B[1][0, 0]*B[1][1, 1]/4 - B[1][0, 1]**2/8 - B[1][0, 1]*B[1][1, 0]/4 - B[1][1, 0]**2/8 + B[1][1, 1]**2/8)*sin(4*phi) + B[1][0, 0]*B[1][1, 0]/8 - B[1][0, 1]*B[1][1, 1]/8